## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Explain Clinical Document Generic

- **Model**: `en.explain_doc.clinical_generic.pipeline`
- **Model Description**:  The pipeline is designed to extract clinical entities, assign assertion status to them, and establish relations between the extracted entities.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-explain-doc-clinical-generic-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [7]:
docs = [
    '''The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI.  She was prescribed 5000 units of Fragmin  subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.''',
    '''The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.'''
]


sample_text = """Patient with severe fever and sore throat.
He shows no stomach pain. He maintained on an epidural and PCA for pain control.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer."""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data["ner_predictions"])
assertion_df = pd.DataFrame(data["assertion_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])

In [9]:
ner_df

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",severe fever,13,24,PROBLEM,0.902
1,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",sore throat,30,40,PROBLEM,0.69635
2,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",stomach pain,55,66,PROBLEM,0.82834995
3,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",an epidural,86,96,TREATMENT,0.96599996
4,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",PCA,102,104,TREATMENT,0.8087
5,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",pain,110,113,PROBLEM,0.9875
6,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",CT,130,131,PROBLEM,0.9934
7,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",lung tumor,134,143,PROBLEM,0.87944996
8,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",Alzheimer,190,198,PROBLEM,0.9796


In [10]:
assertion_df

,document,ner_chunk,begin,end,ner_label,assertion
0,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",severe fever,13,24,PROBLEM,Present
1,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",sore throat,30,40,PROBLEM,Present
2,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",stomach pain,55,66,PROBLEM,Absent
3,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",an epidural,86,96,TREATMENT,Past
4,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",PCA,102,104,TREATMENT,Past
5,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",pain,110,113,PROBLEM,Hypothetical
6,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",CT,130,131,PROBLEM,Past
7,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",lung tumor,134,143,PROBLEM,Present
8,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",Alzheimer,190,198,PROBLEM,Family


In [11]:
relation_df

,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",severe fever,13,24,PROBLEM,sore throat,30,40,PROBLEM,PIP,0.99999845
1,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",an epidural,86,96,TREATMENT,PCA,102,104,TREATMENT,O,0.9999999
2,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",PCA,102,104,TREATMENT,pain,110,113,PROBLEM,TrAP,0.9989556
3,"Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzheimer.",CT,130,131,PROBLEM,lung tumor,134,143,PROBLEM,PIP,0.999998


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [12]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data["ner_predictions"])
assertion_df = pd.DataFrame(data["assertion_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])

In [13]:
ner_df

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,0.5547
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",diabetes,77,84,PROBLEM,0.6342
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",type 2,87,92,PROBLEM,0.96895003
3,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Bactrim,120,126,TREATMENT,0.9916
4,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",UTI,144,146,PROBLEM,0.9903
5,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",5000 units of Fragmin subcutaneously,169,205,TREATMENT,0.59686005
6,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Lantus,229,234,TREATMENT,0.9955
7,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",1 capsule of Advil 10 mg,27,50,TREATMENT,0.63818336
8,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",magnesium hydroxide,67,85,TREATMENT,0.84870005
9,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",40 units of insulin glargine,179,206,TREATMENT,0.61964


In [14]:
assertion_df

,document,ner_chunk,begin,end,ner_label,assertion
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,Family
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",diabetes,77,84,PROBLEM,Present
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",type 2,87,92,PROBLEM,Present
3,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Bactrim,120,126,TREATMENT,Past
4,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",UTI,144,146,PROBLEM,Past
5,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",5000 units of Fragmin subcutaneously,169,205,TREATMENT,Past
6,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Lantus,229,234,TREATMENT,Past
7,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",1 capsule of Advil 10 mg,27,50,TREATMENT,Planned
8,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",magnesium hydroxide,67,85,TREATMENT,Planned
9,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",40 units of insulin glargine,179,206,TREATMENT,Past


In [15]:
relation_df

,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,diabetes,77,84,PROBLEM,TrAP,0.9747736
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,type 2,87,92,PROBLEM,TrAP,1.0
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",diabetes,77,84,PROBLEM,type 2,87,92,PROBLEM,PIP,0.5765517
3,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Bactrim,120,126,TREATMENT,UTI,144,146,PROBLEM,TrAP,0.99997115
4,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",1 capsule of Advil 10 mg,27,50,TREATMENT,magnesium hydroxide,67,85,TREATMENT,O,0.9999919
5,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",40 units of insulin glargine,179,206,TREATMENT,12 units of insulin lispro,218,243,TREATMENT,O,0.8752808


### JSON Lines

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [16]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

In [17]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "insulin", "begin": 59, "end": 65, "ner_label": "TREATMENT", "ner_confidence": "0.5547"}, {"ner_chunk": "diabetes", "begin": 77, "end": 84, "ner_label": "PROBLEM", "ner_confidence": "0.6342"}, {"ner_chunk": "type 2", "begin": 87, "end": 92, "ner_label": "PROBLEM", "ner_confidence": "0.96895003"}, {"ner_chunk": "Bactrim", "begin": 120, "end": 126, "ner_label": "TREATMENT", "ner_confidence": "0.9916"}, {"ner_chunk": "UTI", "begin": 144, "end": 146, "ner_label": "PROBLEM", "ner_confidence": "0.9903"}, {"ner_chunk": "5000 units of Fragmin  subcutaneously", "begin": 169, "end": 205, "ner_label": "TREATMENT", "ner_confidence": "0.59686005"}, {"ner_chunk": "Lantus", "begin": 229, "end": 234, "ner_label": "TREATMENT", "ner_confidence": "0.9955"}], "assertion_predictions": [{"ner_chunk": "insulin", "begin": 59, "end": 65, "ner_label": "TREATMENT", "assertion": "Family"}, {"ner_chunk": "diabetes", "begin": 77, "end": 84, "ner_label": "PROBLEM", "assertion": "Pr

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [18]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [19]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [20]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    data_frames = {
        "NER Predictions DataFrame": pd.DataFrame(data['ner_predictions']),
        "Assertion Predictions DataFrame": pd.DataFrame(data['assertion_predictions']),
        "Relation Predictions DataFrame": pd.DataFrame(data['relation_predictions'])
    }

    # Display and save each DataFrame
    for name, df in data_frames.items():
        print(name + ":")
        display(df)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [23]:
process_s3_json_output_and_save(validation_json_file_name)

NER Predictions DataFrame:


,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,0.5547
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",diabetes,77,84,PROBLEM,0.6342
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",type 2,87,92,PROBLEM,0.96895003
3,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Bactrim,120,126,TREATMENT,0.9916
4,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",UTI,144,146,PROBLEM,0.9903
5,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",5000 units of Fragmin subcutaneously,169,205,TREATMENT,0.59686005
6,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Lantus,229,234,TREATMENT,0.9955
7,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",1 capsule of Advil 10 mg,27,50,TREATMENT,0.63818336
8,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",magnesium hydroxide,67,85,TREATMENT,0.84870005
9,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",40 units of insulin glargine,179,206,TREATMENT,0.61964


Assertion Predictions DataFrame:


,document,ner_chunk,begin,end,ner_label,assertion
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,Family
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",diabetes,77,84,PROBLEM,Present
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",type 2,87,92,PROBLEM,Present
3,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Bactrim,120,126,TREATMENT,Past
4,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",UTI,144,146,PROBLEM,Past
5,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",5000 units of Fragmin subcutaneously,169,205,TREATMENT,Past
6,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Lantus,229,234,TREATMENT,Past
7,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",1 capsule of Advil 10 mg,27,50,TREATMENT,Planned
8,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",magnesium hydroxide,67,85,TREATMENT,Planned
9,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",40 units of insulin glargine,179,206,TREATMENT,Past


Relation Predictions DataFrame:


,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,diabetes,77,84,PROBLEM,TrAP,0.9747736
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",insulin,59,65,TREATMENT,type 2,87,92,PROBLEM,TrAP,1.0
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",diabetes,77,84,PROBLEM,type 2,87,92,PROBLEM,PIP,0.5765517
3,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI. She was prescribed 5000 units of Fragmin subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime.",Bactrim,120,126,TREATMENT,UTI,144,146,PROBLEM,TrAP,0.99997115
4,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",1 capsule of Advil 10 mg,27,50,TREATMENT,magnesium hydroxide,67,85,TREATMENT,O,0.9999919
5,"The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.",40 units of insulin glargine,179,206,TREATMENT,12 units of insulin lispro,218,243,TREATMENT,O,0.8752808


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [25]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [26]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "insulin", "begin": 59, "end": 65, "ner_label": "TREATMENT", "ner_confidence": "0.5547"}, {"ner_chunk": "diabetes", "begin": 77, "end": 84, "ner_label": "PROBLEM", "ner_confidence": "0.6342"}, {"ner_chunk": "type 2", "begin": 87, "end": 92, "ner_label": "PROBLEM", "ner_confidence": "0.96895003"}, {"ner_chunk": "Bactrim", "begin": 120, "end": 126, "ner_label": "TREATMENT", "ner_confidence": "0.9916"}, {"ner_chunk": "UTI", "begin": 144, "end": 146, "ner_label": "PROBLEM", "ner_confidence": "0.9903"}, {"ner_chunk": "5000 units of Fragmin  subcutaneously", "begin": 169, "end": 205, "ner_label": "TREATMENT", "ner_confidence": "0.59686005"}, {"ner_chunk": "Lantus", "begin": 229, "end": 234, "ner_label": "TREATMENT", "ner_confidence": "0.9955"}], "assertion_predictions": [{"ner_chunk": "insulin", "begin": 59, "end": 65, "ner_label": "TREATMENT", "assertion": "Family"}, {"ner_chunk": "diabetes", "begin": 77, "end": 84, "ner_label": "PROBLEM", "assertion": "Pr

In [27]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-generic-pipelin-2024-05-15-21-29-45-296


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

